# Universalbank

In [1]:
# Import the libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandasql as psql 

# Ignore Harmfull warnings

import warnings

warnings.filterwarnings('ignore')

In [2]:
# Load the Universal Bank dataset

bankdata = pd.read_csv(r"D:\iiit notes\Programming\AI\Internship practice\50 season bagging 30-jul-2021\Universalbank.csv",header= 0)
bankdata.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [3]:
# Display the information of dataset

bankdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [4]:
# Delete the varaiables which are not impacting the target variable
del bankdata['ID']
del bankdata['ZIP Code']

In [5]:
# cols is varaiables- creating a dummy variables

cols1 = ['Family', 'Education']

In [6]:
# cols2 variable -MinMaxScaler function

cols2 = ['Age', 'Experience', 'Income', 'CCAvg', 'Mortgage']
print(cols2)

['Age', 'Experience', 'Income', 'CCAvg', 'Mortgage']


In [7]:
# create dummy variable for all range values

bankdata = pd.get_dummies(bankdata, columns=cols1)
bankdata.head().T

,0,1,2,3,4
Age,25.0,45.0,39.0,35.0,35.0
Experience,1.0,19.0,15.0,9.0,8.0
Income,49.0,34.0,11.0,100.0,45.0
CCAvg,1.6,1.5,1.0,2.7,1.0
Mortgage,0.0,0.0,0.0,0.0,0.0
Personal Loan,0.0,0.0,0.0,0.0,0.0
Securities Account,1.0,1.0,0.0,0.0,0.0
CD Account,0.0,0.0,0.0,0.0,0.0
Online,0.0,0.0,0.0,0.0,0.0
CreditCard,0.0,0.0,0.0,0.0,1.0


In [8]:
# Identify the dependent and target variables

IndepVar = []
for col in bankdata.columns:
    if col != 'CreditCard':
        IndepVar.append(col)
        
TargetVar = 'CreditCard'

x = bankdata[IndepVar]
y = bankdata[TargetVar]

In [9]:
# Splitting the dataset into train and test 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state = 42) 
x_test_F1 = x_test.copy()

In [10]:
# Scaling the features by using MinMaxScaler 

from sklearn.preprocessing import MinMaxScaler 

mmscaler = MinMaxScaler(feature_range=(0, 1)) 

x_train[cols2] = mmscaler.fit_transform(x_train[cols2]) 
x_train = pd.DataFrame(x_train) 

x_test[cols2] = mmscaler.fit_transform(x_test[cols2]) 
x_test = pd.DataFrame(x_test)

# Bagging Classifier Algorithm

In [11]:
# Training bagging classifier - BaggingClassifier class of 'sklearn.ensemble' packages to build bagging classifier model. 
# We set DecisionTreeClassifier class as a base estimator and set 100 to the number of estimators, then train the model 
# with train data. 

from sklearn.ensemble import BaggingClassifier 
from sklearn.tree import DecisionTreeClassifier 

modelDT = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, 
                                 min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, 
                                 min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, ccp_alpha=0.0) 

modelBAG = BaggingClassifier(base_estimator=modelDT, n_estimators=100, max_samples=1.0, max_features=1.0, bootstrap=True, 
                             bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=None, random_state=None, 
                             verbose=0) 

modelBAG.fit(x_train,y_train) 

# Predict model with test data 
y_pred = modelBAG.predict(x_test)

# confusion matrix in sklearn 

from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report

# actual values

actual = y_test 

# predicted values

predicted = y_pred

# confusion matrix 
matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None) 

print('Confusion matrix : \n', matrix) 

# outcome values order in sklearn 

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1) 

print('Outcome values : \n', tp, fn, fp, tn) 

# classification report for precision, recall f1-score and accuracy 

matrix = classification_report(actual,predicted,labels=[1,0]) 
print('Classification report : \n',matrix) 

# calculating the metrics 

sensitivity = round(tp/(tp+fn), 3) 
specificity = round(tn/(tn+fp), 3); 

accuracy = round((tp+tn)/(tp+fp+tn+fn), 3); 

balanced_accuracy = round((sensitivity+specificity)/2, 3); 

precision = round(tp/(tp+fp), 3); f1Score = round((2*tp/(2*tp + fp + fn)), 3); 

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model 

from math import sqrt 

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn) 

MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3) 

print('Accuracy :', round(accuracy*100, 2),'%') 
print('Precision :', round(precision*100, 2),'%') 
print('Recall :', round(sensitivity*100,2), '%') 
print('F1 Score :', f1Score) 
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%') 
print('MCC :', MCC) 

# Area under ROC curve 

from sklearn.metrics  import roc_curve, roc_auc_score 
print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))


Confusion matrix : 
 [[112 315]
 [117 956]]
Outcome values : 
 112 315 117 956
Classification report : 
               precision    recall  f1-score   support

           1       0.49      0.26      0.34       427
           0       0.75      0.89      0.82      1073

    accuracy                           0.71      1500
   macro avg       0.62      0.58      0.58      1500
weighted avg       0.68      0.71      0.68      1500

Accuracy : 71.2 %
Precision : 48.9 %
Recall : 26.2 %
F1 Score : 0.341
Balanced Accuracy : 57.6 %
MCC : 0.192
roc_auc_score: 0.577


# Using Bagging Classifier for All out Algorithm

In [12]:
# Checking accuracy by changing base estimator - 

from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import BaggingClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import GradientBoostingClassifier 

modelLR = LogisticRegression() 
modelRF = RandomForestClassifier(criterion='gini', n_estimators=500, random_state=0) 
modelDT = DecisionTreeClassifier(criterion="entropy") 
modelSVMGaussian = SVC(kernel='rbf', random_state = None, class_weight=None,probability=True) 
modelKNN = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', 
                                metric_params=None, n_jobs=None) 
modelXGB = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, 
                                      criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, 
                                      min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, 
                                      min_impurity_split=None, init=None, random_state=None, max_features=None, 
                                      verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, 
                                      n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
base_methods=[None, modelLR, modelRF, modelDT, modelSVMGaussian, modelKNN, modelXGB] 
for bm in base_methods: 
    print("Method: ", bm) 
    modelBAG = BaggingClassifier(base_estimator=bm,n_estimators=100,bootstrap=True) 
    modelBAG.fit(x_train, y_train) 
    y1_pred = modelBAG.predict(x_test)
    
    # confusion matrix in sklearn 
    from sklearn.metrics import confusion_matrix 
    from sklearn.metrics import classification_report 
    
    # actual values 
    
    actual = y_test 
    
    # predicted values
    
    predicted = y1_pred 
    
    # confusion matrix
    
    matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
    print('Confusion matrix : \n', matrix) 
    
    # outcome values order in sklearn 
    
    tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1) 
    print('Outcome values : \n', tp, fn, fp, tn) 
    # classification report for precision, recall f1-score and accuracy 
    
    matrix = classification_report(actual,predicted,labels=[1,0]) 
    
    print('Classification report : \n',matrix) 
    
    # calculating the metrics 
    
    sensitivity = round(tp/(tp+fn), 3) 
    specificity = round(tn/(tn+fp), 3); 
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3); 
    balanced_accuracy = round((sensitivity+specificity)/2, 3); 
    precision = round(tp/(tp+fp), 3); 
    f1Score = round((2*tp/(2*tp + fp + fn)), 3); 
    
    # Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
    # A model with a score of +1 is a perfect model and -1 is a poor model 
    
    from math import sqrt 
    
    x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn) 

    MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3) 
    
    print('Accuracy :', round(accuracy*100, 2),'%') 
    print('Precision :', round(precision*100, 2),'%') 
    print('Recall :', round(sensitivity*100,2), '%') 
    print('F1 Score :', f1Score) 
    print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%') 
    print('MCC :', MCC) 
    
    # Area under ROC curve 
    
    from sklearn.metrics import roc_auc_score
    print('roc_auc_score:', round(roc_auc_score(y_test, y1_pred), 3)) 
    print('-----------------------------------------------------------------------')

Method:  None
Confusion matrix : 
 [[115 312]
 [119 954]]
Outcome values : 
 115 312 119 954
Classification report : 
               precision    recall  f1-score   support

           1       0.49      0.27      0.35       427
           0       0.75      0.89      0.82      1073

    accuracy                           0.71      1500
   macro avg       0.62      0.58      0.58      1500
weighted avg       0.68      0.71      0.68      1500

Accuracy : 71.3 %
Precision : 49.1 %
Recall : 26.9 %
F1 Score : 0.348
Balanced Accuracy : 57.9 %
MCC : 0.197
roc_auc_score: 0.579
-----------------------------------------------------------------------
Method:  LogisticRegression()
Confusion matrix : 
 [[  71  356]
 [  17 1056]]
Outcome values : 
 71 356 17 1056
Classification report : 
               precision    recall  f1-score   support

           1       0.81      0.17      0.28       427
           0       0.75      0.98      0.85      1073

    accuracy                           0.75      1

# SVM Algorithm - Gaussian Kernel

In [13]:
# train the SVM algorithm

from sklearn.svm import SVC

modelSVMGaussian = SVC(kernel='rbf', random_state = None, class_weight=None, probability=True)

modelSVMGaussian.fit(x_train, y_train)

# Predict the values

y2_pred = modelSVMGaussian.predict(x_test)

# Confusion matrix and classification report

from sklearn.metrics import classification_report, confusion_matrix

# Actual values

Actual = y_test

# Predicted Values

Predicted = y2_pred

# confusion matrix 

matrix = confusion_matrix(Actual,Predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix) 

# Outcome values in order

tp, fn, fp, tn = confusion_matrix(Actual,Predicted,labels=[1,0]).reshape(-1) 
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall, f1-score

matrix = classification_report(Actual,Predicted,labels=[1,0]) 
print('Classifiction_report : \n', matrix)

# Calculate the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3)
accuarcy = round((tp+tn)/(tp+fn+fp+tn), 3)
balanced_accuracy = round((sensitivity+specificity)/2, 3)
precision =round(tp/(tp+fp), 3)
f1score = round((2*tp/(2*tp + fp + fn)), 3)

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp*fn) + (tp*fp) + (tn*fn) + (tn*fp)

MCC = round(((tp*tn) - (fp*fn))/sqrt(x), 3)

print('Accuracy: ', accuracy*100, '%')
print('Recall: ', sensitivity*100, '%')
print('Precision: ', precision*100, '%')
print('f1score: ', f1score)
print('Balanced Accuracy: ', balanced_accuracy*100, '%')
print('MCC: ', MCC)

# Area under ROC curve 
    
print('roc_auc_score:', round(roc_auc_score(y_test, y2_pred), 3)) 
print('-----------------------------------------------------------------------')

Confusion matrix : 
 [[  67  360]
 [  10 1063]]
Outcome values : 
 67 360 10 1063
Classifiction_report : 
               precision    recall  f1-score   support

           1       0.87      0.16      0.27       427
           0       0.75      0.99      0.85      1073

    accuracy                           0.75      1500
   macro avg       0.81      0.57      0.56      1500
weighted avg       0.78      0.75      0.68      1500

Accuracy:  75.2 %
Recall:  15.7 %
Precision:  87.0 %
f1score:  0.266
Balanced Accuracy:  57.4 %
MCC:  104.578
roc_auc_score: 0.574
-----------------------------------------------------------------------


# Logistic Regression

In [14]:
# Train the data with Logistic Regression and with random sampling

from sklearn.linear_model import LogisticRegression

modelLR = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, 
                             class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', 
                             verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
modelLR.fit(x_train, y_train)

# Predice the model with test dataset

y3_pred = modelLR.predict(x_test)

# Confusion matrix and classification report

from sklearn.metrics import classification_report, confusion_matrix

# Actual values

Actual = y_test

# Predicted Values

Predicted = y3_pred

# confusion matrix 

matrix = confusion_matrix(Actual,Predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix) 

# Outcome values in order

tp, fn, fp, tn = confusion_matrix(Actual,Predicted,labels=[1,0]).reshape(-1) 
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall, f1-score

matrix = classification_report(Actual,Predicted,labels=[1,0]) 
print('Classifiction_report : \n', matrix)

# Calculate the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3)
accuarcy = round((tp+tn)/(tp+fn+fp+tn), 3)
balanced_accuracy = round((sensitivity+specificity)/2, 3)
precision =round(tp/(tp+fp), 3)
f1score = round((2*tp/(2*tp + fp + fn)), 3)

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp*fn) + (tp*fp) + (tn*fn) + (tn*fp)

MCC = round(((tp*tn) - (fp*fn))/sqrt(x), 3)

print('Accuracy: ', accuracy*100, '%')
print('Recall: ', sensitivity*100, '%')
print('Precision: ', precision*100, '%')
print('f1score: ', f1score)
print('Balanced Accuracy: ', balanced_accuracy*100, '%')
print('MCC: ', MCC)

# Area under ROC curve 
    
print('roc_auc_score:', round(roc_auc_score(y_test, y3_pred), 3)) 
print('-----------------------------------------------------------------------')

Confusion matrix : 
 [[  71  356]
 [  17 1056]]
Outcome values : 
 71 356 17 1056
Classifiction_report : 
               precision    recall  f1-score   support

           1       0.81      0.17      0.28       427
           0       0.75      0.98      0.85      1073

    accuracy                           0.75      1500
   macro avg       0.78      0.58      0.56      1500
weighted avg       0.76      0.75      0.69      1500

Accuracy:  75.2 %
Recall:  16.6 %
Precision:  80.7 %
f1score:  0.276
Balanced Accuracy:  57.49999999999999 %
MCC:  106.305
roc_auc_score: 0.575
-----------------------------------------------------------------------


# Decision Tree with gini

In [15]:
# Train the model and build the 'Decision Tree' Algorithm

from sklearn.tree import DecisionTreeClassifier

modelDTC = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, 
                                  min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, 
                                  min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, ccp_alpha=0.0)
# Train the model

modelDTC.fit(x_train, y_train)

# Predict the model with test dataset

y4_pred = modelDTC.predict(x_test)

# Confusion matrix and classification report

from sklearn.metrics import classification_report, confusion_matrix

# Actual values

Actual = y_test

# Predicted Values

Predicted = y4_pred

# confusion matrix 

matrix = confusion_matrix(Actual,Predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix) 

# Outcome values in order

tp, fn, fp, tn = confusion_matrix(Actual,Predicted,labels=[1,0]).reshape(-1) 
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall, f1-score

matrix = classification_report(Actual,Predicted,labels=[1,0]) 
print('Classifiction_report : \n', matrix)

# Calculate the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3)
accuarcy = round((tp+tn)/(tp+fn+fp+tn), 3)
balanced_accuracy = round((sensitivity+specificity)/2, 3)
precision =round(tp/(tp+fp), 3)
f1score = round((2*tp/(2*tp + fp + fn)), 3)

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp*fn) + (tp*fp) + (tn*fn) + (tn*fp)

MCC = round(((tp*tn) - (fp*fn))/sqrt(x), 3)

print('Accuracy: ', accuracy*100, '%')
print('Recall: ', sensitivity*100, '%')
print('Precision: ', precision*100, '%')
print('f1score: ', f1score)
print('Balanced Accuracy: ', balanced_accuracy*100, '%')
print('MCC: ', MCC)

# Area under ROC curve 
    
print('roc_auc_score:', round(roc_auc_score(y_test, y4_pred), 3)) 
print('-----------------------------------------------------------------------')

Confusion matrix : 
 [[173 254]
 [291 782]]
Outcome values : 
 173 254 291 782
Classifiction_report : 
               precision    recall  f1-score   support

           1       0.37      0.41      0.39       427
           0       0.75      0.73      0.74      1073

    accuracy                           0.64      1500
   macro avg       0.56      0.57      0.56      1500
weighted avg       0.65      0.64      0.64      1500

Accuracy:  75.2 %
Recall:  40.5 %
Precision:  37.3 %
f1score:  0.388
Balanced Accuracy:  56.699999999999996 %
MCC:  85.069
roc_auc_score: 0.567
-----------------------------------------------------------------------


# Random Forest

In [16]:
# Build Random Forest classification model and Train the model using the training sets 

from sklearn.ensemble import RandomForestClassifier 

modelRF = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None, criterion='entropy', max_depth=None, 
                                 max_features='auto', max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, 
                                 min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, 
                                 min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None, oob_score=False, 
                                 random_state=0, verbose=0, warm_start=False) 
modelRF = modelRF.fit(x_train, y_train) 

# Predict the model with test data set 

y5_pred = modelRF.predict(x_test) 

# Confusion matrix and classification report

from sklearn.metrics import classification_report, confusion_matrix

# Actual values

Actual = y_test

# Predicted Values

Predicted = y5_pred

# confusion matrix 

matrix = confusion_matrix(Actual,Predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix) 

# Outcome values in order

tp, fn, fp, tn = confusion_matrix(Actual,Predicted,labels=[1,0]).reshape(-1) 
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall, f1-score

matrix = classification_report(Actual,Predicted,labels=[1,0]) 
print('Classifiction_report : \n', matrix)

# Calculate the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3)
accuarcy = round((tp+tn)/(tp+fn+fp+tn), 3)
balanced_accuracy = round((sensitivity+specificity)/2, 3)
precision =round(tp/(tp+fp), 3)
f1score = round((2*tp/(2*tp + fp + fn)), 3)

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp*fn) + (tp*fp) + (tn*fn) + (tn*fp)

MCC = round(((tp*tn) - (fp*fn))/sqrt(x), 3)

print('Accuracy: ', accuracy*100, '%')
print('Recall: ', sensitivity*100, '%')
print('Precision: ', precision*100, '%')
print('f1score: ', f1score)
print('Balanced Accuracy: ', balanced_accuracy*100, '%')
print('MCC: ', MCC)

# Area under ROC curve 
    
print('roc_auc_score:', round(roc_auc_score(y_test, y5_pred), 3)) 
print('-----------------------------------------------------------------------')

Confusion matrix : 
 [[112 315]
 [109 964]]
Outcome values : 
 112 315 109 964
Classifiction_report : 
               precision    recall  f1-score   support

           1       0.51      0.26      0.35       427
           0       0.75      0.90      0.82      1073

    accuracy                           0.72      1500
   macro avg       0.63      0.58      0.58      1500
weighted avg       0.68      0.72      0.68      1500

Accuracy:  75.2 %
Recall:  26.200000000000003 %
Precision:  50.7 %
f1score:  0.346
Balanced Accuracy:  57.99999999999999 %
MCC:  109.014
roc_auc_score: 0.58
-----------------------------------------------------------------------


# KNN ALgorithm

In [17]:
# To build the 'KNN' model 
from sklearn.neighbors import KNeighborsClassifier 

modelKNN = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', 
                                metric_params=None, n_jobs=None) 
modelKNN = modelKNN.fit(x_train, y_train) 

# Predict the model with test data set 
y6_pred = modelKNN.predict(x_test) 

# Confusion matrix and classification report

from sklearn.metrics import classification_report, confusion_matrix

# Actual values

Actual = y_test

# Predicted Values

Predicted = y6_pred

# confusion matrix 

matrix = confusion_matrix(Actual,Predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix) 

# Outcome values in order

tp, fn, fp, tn = confusion_matrix(Actual,Predicted,labels=[1,0]).reshape(-1) 
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall, f1-score

matrix = classification_report(Actual,Predicted,labels=[1,0]) 
print('Classifiction_report : \n', matrix)

# Calculate the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3)
accuarcy = round((tp+tn)/(tp+fn+fp+tn), 3)
balanced_accuracy = round((sensitivity+specificity)/2, 3)
precision =round(tp/(tp+fp), 3)
f1score = round((2*tp/(2*tp + fp + fn)), 3)

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp*fn) + (tp*fp) + (tn*fn) + (tn*fp)

MCC = round(((tp*tn) - (fp*fn))/sqrt(x), 3)

print('Accuracy: ', accuracy*100, '%')
print('Recall: ', sensitivity*100, '%')
print('Precision: ', precision*100, '%')
print('f1score: ', f1score)
print('Balanced Accuracy: ', balanced_accuracy*100, '%')
print('MCC: ', MCC)

# Area under ROC curve 
    
print('roc_auc_score:', round(roc_auc_score(y_test, y6_pred), 3)) 
print('-----------------------------------------------------------------------')

Confusion matrix : 
 [[127 300]
 [161 912]]
Outcome values : 
 127 300 161 912
Classifiction_report : 
               precision    recall  f1-score   support

           1       0.44      0.30      0.36       427
           0       0.75      0.85      0.80      1073

    accuracy                           0.69      1500
   macro avg       0.60      0.57      0.58      1500
weighted avg       0.66      0.69      0.67      1500

Accuracy:  75.2 %
Recall:  29.7 %
Precision:  44.1 %
f1score:  0.355
Balanced Accuracy:  57.4 %
MCC:  97.566
roc_auc_score: 0.574
-----------------------------------------------------------------------


# Gradient Boosting

In [18]:
# To build the 'Gradient Boosting' model 
from sklearn.ensemble import GradientBoostingClassifier 
modelXGB = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, 
                                      criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, 
                                      min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, 
                                      min_impurity_split=None, init=None, random_state=None, max_features=None, 
                                      verbose=0, max_leaf_nodes=None, warm_start=False, 
                                      validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0) 
modelXGB = modelXGB.fit(x_train,y_train) 

# Predict the model with test data set 

y7_pred = modelXGB.predict(x_test) 

# Confusion matrix and classification report

from sklearn.metrics import classification_report, confusion_matrix

# Actual values

Actual = y_test

# Predicted Values

Predicted = y7_pred

# confusion matrix 

matrix = confusion_matrix(Actual,Predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix) 

# Outcome values in order

tp, fn, fp, tn = confusion_matrix(Actual,Predicted,labels=[1,0]).reshape(-1) 
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall, f1-score

matrix = classification_report(Actual,Predicted,labels=[1,0]) 
print('Classifiction_report : \n', matrix)

# Calculate the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3)
accuarcy = round((tp+tn)/(tp+fn+fp+tn), 3)
balanced_accuracy = round((sensitivity+specificity)/2, 3)
precision =round(tp/(tp+fp), 3)
f1score = round((2*tp/(2*tp + fp + fn)), 3)

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp*fn) + (tp*fp) + (tn*fn) + (tn*fp)

MCC = round(((tp*tn) - (fp*fn))/sqrt(x), 3)

print('Accuracy: ', accuracy*100, '%')
print('Recall: ', sensitivity*100, '%')
print('Precision: ', precision*100, '%')
print('f1score: ', f1score)
print('Balanced Accuracy: ', balanced_accuracy*100, '%')
print('MCC: ', MCC)

# Area under ROC curve 
    
print('roc_auc_score:', round(roc_auc_score(y_test, y7_pred), 3)) 
print('-----------------------------------------------------------------------')

Confusion matrix : 
 [[  72  355]
 [  19 1054]]
Outcome values : 
 72 355 19 1054
Classifiction_report : 
               precision    recall  f1-score   support

           1       0.79      0.17      0.28       427
           0       0.75      0.98      0.85      1073

    accuracy                           0.75      1500
   macro avg       0.77      0.58      0.56      1500
weighted avg       0.76      0.75      0.69      1500

Accuracy:  75.2 %
Recall:  16.900000000000002 %
Precision:  79.10000000000001 %
f1score:  0.278
Balanced Accuracy:  57.599999999999994 %
MCC:  106.547
roc_auc_score: 0.575
-----------------------------------------------------------------------
